In [13]:
import os
import sys
from pathlib import Path

def setup_project_path():
    current = Path.cwd()
    while not (current / 'craft').exists():
        current = current.parent
    return current
project_root = setup_project_path()
sys.path.insert(0, str(project_root))

In [23]:
def compute_letter_spacings_with_text(centers, text):
    """
    centers : [(cx, cy), ...]  # 공백 제외 글자들의 중심점
    text    : 원본 문자열 (띄어쓰기 포함)

    return  : spacing_items 리스트
        spacing_items = [
            (i, i+1, spacing, is_word_gap), ...
        ]

        - i, i+1 : x 정렬된 centers 상에서의 인접 인덱스
        - spacing: x2 - x1 (수평 거리, pixel 단위)
        - is_word_gap:
            False → 단어 내부 자간 (공백 없음)
            True  → 단어 사이 공백(띄어쓰기) 구간
    """
    spacing_items = []

    if len(centers) < 2:
        return spacing_items

    # 1) text에서 공백 제외 문자 인덱스
    non_space_indices = [idx for idx, ch in enumerate(text) if ch != " "]

    if len(non_space_indices) != len(centers):
        raise ValueError(
            "텍스트(공백 제외) 글자 수와 center 개수가 다릅니다: "
            "text_non_space=%d, centers=%d"
            % (len(non_space_indices), len(centers))
        )

    # 2) x 기준 정렬 (기울기 계산할 때와 같은 기준)
    centers_sorted = sorted(centers, key=lambda p: p[0])

    # 3) 인접 쌍마다 spacing 계산
    for i in range(len(centers_sorted) - 1):
        x1, y1 = centers_sorted[i]
        x2, y2 = centers_sorted[i + 1]

        idx1 = non_space_indices[i]
        idx2 = non_space_indices[i + 1]

        # idx1~idx2 사이에 공백이 있으면 단어 사이 간격
        segment = text[idx1 + 1 : idx2 + 1]
        has_space = (" " in segment)  # True면 단어 사이 공백

        spacing = x2 - x1  # 수평 거리(px)

        spacing_items.append((i, i + 1, spacing, has_space))

    return spacing_items


In [15]:
def load_centers_from_file(centers_file: str):
    """
    centers_file: res_{filename}_center.txt
    return: [(cx, cy), ...]
    """
    centers = []
    if not os.path.exists(centers_file):
        raise FileNotFoundError(f"centers file not found: {centers_file}")

    with open(centers_file, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            # 포맷: cx,cy
            x_str, y_str = line.split(",")
            centers.append((float(x_str), float(y_str)))
    return centers

In [19]:
def analyze_spacing_from_centers_files(
    result_folder: str,
    spacing_folder: str,
    text_map: dict,
):
    """
    result_folder:
      - res_{filename}_center.txt 가 존재하는 폴더

    spacing_folder:
      - res_{filename}_spacing.txt 를 저장할 폴더

    text_map:
      - {"test1": "안 녕 하 세 요", ...}
    """
    os.makedirs(spacing_folder, exist_ok=True)

    for fname in os.listdir(result_folder):
        if not fname.startswith("res_") or not fname.endswith("_center.txt"):
            continue

        # res_{filename}_center.txt → filename
        core = fname[len("res_") : -len("_center.txt")]

        if core not in text_map:
            print(f"[WARN] text_map에 '{core}' 원문이 없어 자간 파일을 생성하지 않습니다.")
            continue

        centers_file = os.path.join(result_folder, fname)
        original_text = text_map[core]

        try:
            centers = load_centers_from_file(centers_file)
            spacing_items = compute_letter_spacings_with_text(
                centers=centers,
                text=original_text,
            )
        except Exception as e:
            print(f"[WARN] '{core}' 자간 계산 중 오류 발생: {e}")
            continue

        if not spacing_items:
            print(f"[INFO] '{core}' spacing_items가 비어있어 저장하지 않습니다.")
            continue

        spacing_file = os.path.join(spacing_folder, f"res_{core}_spacing.txt")
        with open(spacing_file, "w", encoding="utf-8") as f:
            for (i1, i2, spacing, is_word_gap) in spacing_items:
                gap_flag = "gap" if is_word_gap else "normal"
                f.write(f"{i1},{i2},{spacing:.2f},{gap_flag}\n")

        print(f"[INFO] spacing 저장: {spacing_file}")


In [29]:
text_map = {
    "test1": "안녕하세요",
    "test2": "안녕하세요",
    "test3": "안녕하세요",
    "test4": "안녕하세요",
    "test5": "다람쥐 헌 챗바퀴에 타고파",
}

analyze_spacing_from_centers_files(
    result_folder=str(project_root/"craft"/"results"/"center"),
    spacing_folder=str(project_root/"craft"/"results"/"spacing_module"),
    text_map=text_map,
)



[INFO] spacing 저장: D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\craft\results\spacing_module\res_test1_spacing.txt
[INFO] spacing 저장: D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\craft\results\spacing_module\res_test2_spacing.txt
[INFO] spacing 저장: D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\craft\results\spacing_module\res_test3_spacing.txt
[INFO] spacing 저장: D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\craft\results\spacing_module\res_test4_spacing.txt
[INFO] spacing 저장: D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\craft\results\spacing_module\res_test5_spacing.txt
